In [1]:
import numpy as np
import pandas as pd
import pylab as plt
from tqdm.auto import tqdm
from astropy.io import ascii

## 1. See how many spectra (%) can be opened by Astropy

In [2]:
# Read spectra metadata
spectra_info = pd.read_csv("../1. download ALL wise data/wiserep_spectra_combined.csv")

# keep p60 only
spectra_info = spectra_info[spectra_info["Telescope"] == "P60"]

In [3]:
spectra_info["Obj. Type"].value_counts()

Obj. Type
SN Ia                4769
SN II                 782
SN Ic                  92
SN Ia-91T-like         90
SN IIn                 62
SN IIb                 54
SN Ib                  53
SN Ic-BL               52
SN IIP                 51
SN Ia-91bg-like        31
TDE                    30
SN Ia-pec              27
SN                     22
CV                     20
Nova                   18
SLSN-II                15
SN Ib/c                15
SN Ibn                 14
SN I                   14
SLSN-I                 13
AGN                    12
SN Icn                 10
SN Iax[02cx-like]       9
SN Ia-CSM               9
Galaxy                  3
SN II-pec               3
QSO                     3
SN Ib-Ca-rich           2
ILRT                    1
SN IIL                  1
LBV                     1
SN Ic-Ca-rich           1
SN Ic-pec               1
Computed-Ia             1
FBOT                    1
SN Ia-SC                1
Name: count, dtype: int64

In [4]:
filenames = spectra_info["Ascii file"].values

In [6]:
# most efficient way to iterate
# https://stackoverflow.com/questions/7837722/what-is-the-most-efficient-way-to-loop-through-dataframes-with-pandas

for fn, wl_unit, spec_unit, flux_ucoeff, lambda_min, lambda_max, del_lambda in zip(
    spectra_info["Ascii file"],
    spectra_info["WL Units"],
    spectra_info["Spec. units"],
    spectra_info["Flux Unit Coefficient"],
    spectra_info["Lambda-min"],
    spectra_info["Lambda-max"],
    spectra_info["Del-Lambda"],
):
    break

In [7]:
df = ascii.read(f"../1. download ALL wise data/wiserep_data/spectra/{fn}").to_pandas()

In [32]:
spectra_info["Spec. units"].value_counts().index[0]

'erg cm(-2) sec(-1) Ang(-1)'

In [8]:
handcrafted_wl_min = 999 # angstrom
handcrafted_wl_max = 15_000 # angstrom

def check_wavelength_increasing(wavelengths):
    # wavelenths is a 1d array
    is_increasing = np.all(np.diff(wavelengths) > 0)
    return is_increasing

def check_flux_positive(flux):
    # flux is a 1d array
    is_positive = np.all(flux > 0)
    return is_positive


def check_df(df, wl_unit, spec_unit, lambda_min, lambda_max, del_lambda):
    # Check no. of readings make sense
    n_samples = int((lambda_max - lambda_min) // del_lambda)
    assert n_samples == df.shape[0]

    if df.shape[1] == 2:
        check_df2cols(
            df, wl_unit, spec_unit, lambda_min, lambda_max, del_lambda, n_samples
        )
    else:
        pass
        # Raise warning not supported yet


def check_df2cols(df, wl_unit, spec_unit, lambda_min, lambda_max, del_lambda):
    wavelength, flux = df.iloc[:, 0].to_numpy(), df.iloc[:, 1].to_numpy()

    # Check wavelength range is correct
    assert wavelength[0] == lambda_min
    assert wavelength[-1] == lambda_max

    # Check wavelength is monotonically increasing
    assert check_wavelength_increasing(wavelength)

    # Check wavelength units are in angstrom and if not, convert
    if wl_unit!="Angstrom":
        if wl_unit=="Micrometre":
            wavelength=wavelength*10_000 #convert to angstrom
        if wl_unit=="nm:
            wavelength=wavelength*10 #convert to angstrom
        else:
            raise NotImplementedError # add message and mention wl_unit

    # Check wavelength is in a range that make sense
    assert wavelength[0] > handcrafted_wl_min
    assert wavelength[-1] < handcrafted_wl_max

    # Check flux units and convert 
    if spec_unit!="erg cm(-2) sec(-1) Ang(-1)":
        # if spec_unit=="mJy":
        #     flux = ### 
        # else:
            # raise NotImplementedError # add message and mention spec_unit
        raise NotImplementedError # add message and mention spec_unit
    
    # Check if flux are all positive
    assert check_flux_positive(flux)    

,TELESCOPE:,P60
0,3806.7,4.013300e-16
1,3822.6,5.601300e-16
2,3838.6,3.514100e-16
3,3854.7,4.781600e-17
4,3870.8,1.581700e-16
...,...,...
207,9034.6,1.118400e-16
208,9072.4,1.216200e-16
209,9110.4,1.166500e-16
210,9148.5,1.249800e-16


In [34]:
import astropy.units as u
import astropy.constants as const
from astropy.units import equivalencies

# Define the flux in mJy
flux_mJy = 1.0 * u.mJy  # Example value of 1 mJy

# To convert between frequency and wavelength units, we need to specify the wavelength
wavelength = 5500 * u.AA  # Example: optical wavelength at 5500 Angstroms

# Create the equivalency between frequency and wavelength flux densities
# This accounts for the non-linear relationship between frequency and wavelength
fnu_to_flambda = u.spectral_density(wavelength)

# Do the conversion
flux_flambda = flux_mJy.to(u.erg / u.s / u.cm**2 / u.AA, equivalencies=fnu_to_flambda)

print(f"Flux density at {wavelength}:")
print(f"{flux_mJy} = {flux_flambda}")

Flux density at 5500.0 Angstrom:
1.0 mJy = 9.910494479338844e-16 erg / (Angstrom s cm2)
